# RDE (Regime Detection Engine) Training

Training the Transformer + VAE architecture for market regime detection.
This notebook implements the unsupervised learning approach for regime classification.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add src to path
sys.path.append('../src')
from agents.rde.model import RegimeDetectionEngine
from data.matrix_assembler import MatrixAssembler
from data.market_data import MarketDataHandler

In [ ]:
# Configuration
config = {
    'input_dim': 155,
    'hidden_dim': 256,
    'latent_dim': 8,
    'num_heads': 8,
    'num_layers': 3,
    'learning_rate': 1e-4,
    'batch_size': 32,
    'epochs': 200,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

print(f"Training on device: {config['device']}")
device = torch.device(config['device'])

In [ ]:
# Initialize model
model = RegimeDetectionEngine(
    input_dim=config['input_dim'],
    hidden_dim=config['hidden_dim'],
    latent_dim=config['latent_dim'],
    num_heads=config['num_heads'],
    num_layers=config['num_layers']
).to(device)

optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# VAE Loss Function
def vae_loss(outputs, targets, beta=0.1):
    """Combined reconstruction + KL divergence loss"""
    recon_loss = nn.functional.mse_loss(outputs['reconstructed'], targets)
    kl_loss = -0.5 * torch.sum(1 + outputs['log_var'] - outputs['mu'].pow(2) - outputs['log_var'].exp())
    return recon_loss + beta * kl_loss, {'recon': recon_loss.item(), 'kl': kl_loss.item()}

def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    total_recon = 0
    total_kl = 0
    
    for batch_idx, (data,) in enumerate(dataloader):
        data = data.to(device)
        optimizer.zero_grad()
        
        outputs = model(data)
        loss, metrics = vae_loss(outputs, data)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_recon += metrics['recon']
        total_kl += metrics['kl']
    
    return {
        'loss': total_loss / len(dataloader),
        'recon_loss': total_recon / len(dataloader),
        'kl_loss': total_kl / len(dataloader)
    }

In [ ]:
# Generate synthetic training data
def create_training_data(n_samples=10000, sequence_length=100, input_dim=155):
    """Create synthetic MMD sequences for training"""
    # Generate synthetic market regimes
    data = []
    
    for _ in range(n_samples):
        # Simulate different market regimes
        regime_type = np.random.choice(['trending', 'ranging', 'volatile', 'transition'])
        
        if regime_type == 'trending':
            base = np.cumsum(np.random.normal(0.001, 0.01, sequence_length))
        elif regime_type == 'ranging':
            base = np.sin(np.linspace(0, 4*np.pi, sequence_length)) * 0.02
        elif regime_type == 'volatile':
            base = np.random.normal(0, 0.05, sequence_length)
        else:  # transition
            base = np.concatenate([
                np.random.normal(0, 0.01, sequence_length//2),
                np.random.normal(0, 0.03, sequence_length//2)
            ])
        
        # Add features
        features = np.random.randn(sequence_length, input_dim-1) * 0.1
        sequence = np.column_stack([base, features])
        data.append(sequence)
    
    return torch.FloatTensor(np.array(data))

# Create dataset
train_data = create_training_data(n_samples=8000)
val_data = create_training_data(n_samples=2000)

train_dataset = TensorDataset(train_data)
val_dataset = TensorDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

print(f"Training data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")

In [ ]:
# Training loop
train_losses = []
val_losses = []
best_val_loss = float('inf')

for epoch in range(config['epochs']):
    # Train
    train_metrics = train_epoch(model, train_loader, optimizer, device)
    train_losses.append(train_metrics['loss'])
    
    # Validate
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for (data,) in val_loader:
            data = data.to(device)
            outputs = model(data)
            loss, _ = vae_loss(outputs, data)
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../models/rde_best.pth')
    
    # Print progress
    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d}: Train Loss: {train_metrics['loss']:.6f}, "
              f"Val Loss: {val_loss:.6f}, "
              f"Recon: {train_metrics['recon_loss']:.6f}, "
              f"KL: {train_metrics['kl_loss']:.6f}")

print(f"\nTraining completed! Best validation loss: {best_val_loss:.6f}")

In [ ]:
# Plot training curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('RDE Training Progress')

plt.subplot(1, 2, 2)
# Test regime detection
model.eval()
with torch.no_grad():
    sample_data = val_data[:10].to(device)
    outputs = model(sample_data)
    regime_vectors = outputs['regime_vector'].cpu().numpy()
    
    plt.imshow(regime_vectors.T, aspect='auto', cmap='viridis')
    plt.xlabel('Sample')
    plt.ylabel('Regime Dimension')
    plt.title('Regime Vectors')
    plt.colorbar()

plt.tight_layout()
plt.show()

print("\n✅ RDE Training Complete!")
print("Model saved to: ../models/rde_best.pth")
print("Next: Train M-RMS agents")